Full pandas dataframe created for all shep race results

todo:
create graph of box number vs place
create lin model of distance box vs time/place?

idea:
clustering analysis of dog to predict dogs that will behave the same  

working:
currently working with all split time correct in r
creating dog index in r to hold previous race results


In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import tqdm
import pickle

In [2]:
#Loading race results database
os.chdir(r'C:\Users\Nick\Documents\GitHub\grvmodel\DATA\GRV_scraper\full race results')
file_list = os.listdir()
path = os.getcwd()


file_list_new = [x for x in file_list if ".xml" in x]
file_list_new

['rr-BAL01Apr17.xml',
 'rr-BAL01Apr19.xml',
 'rr-BAL01Apr22.xml',
 'rr-BAL01Aug16.xml',
 'rr-BAL01Aug18.xml',
 'rr-BAL01Aug22.xml',
 'rr-BAL01Dec18.xml',
 'rr-BAL01Feb16.xml',
 'rr-BAL01Feb17.xml',
 'rr-BAL01Feb21.xml',
 'rr-BAL01Jan18.xml',
 'rr-BAL01Jan20.xml',
 'rr-BAL01Jul19.xml',
 'rr-BAL01Jun22.xml',
 'rr-BAL01Mar17.xml',
 'rr-BAL01Mar21.xml',
 'rr-BAL01May17.xml',
 'rr-BAL01Nov17.xml',
 'rr-BAL01Nov21.xml',
 'rr-BAL01Nov22.xml',
 'rr-BAL01Oct18.xml',
 'rr-BAL01Sep19.xml',
 'rr-BAL01Sep21.xml',
 'rr-BAL02Apr18.xml',
 'rr-BAL02Aug19.xml',
 'rr-BAL02Aug21.xml',
 'rr-BAL02Dec16.xml',
 'rr-BAL02Dec17.xml',
 'rr-BAL02Dec19.xml',
 'rr-BAL02Feb22.xml',
 'rr-BAL02Jan17.xml',
 'rr-BAL02Jan19.xml',
 'rr-BAL02Jul18.xml',
 'rr-BAL02Jun21.xml',
 'rr-BAL02Mar22.xml',
 'rr-BAL02May17.xml',
 'rr-BAL02Nov16.xml',
 'rr-BAL02Oct17.xml',
 'rr-BAL02Oct19.xml',
 'rr-BAL02Oct21.xml',
 'rr-BAL03Apr17.xml',
 'rr-BAL03Apr19.xml',
 'rr-BAL03Aug16.xml',
 'rr-BAL03Aug17.xml',
 'rr-BAL03Aug22.xml',
 'rr-BAL03

In [3]:
def result_labels(file_list, path):
    label_list = []
    for i in file_list:
        root = ET.parse(i).getroot()
        
        for race in root.iter('Race'):
            race_id = race.attrib["id"]
            print("race id = ",race_id)
            
            places = []
                        
            for dog in race.iter('Dog'):
                places.append(dog[0].text)
            print(places)
        break
            

In [8]:
def result_getter(file_list, path):
    master_list = []

    for i in tqdm.tqdm(file_list):
        root = ET.parse(i).getroot()

        track = root[0]
        date = root[1]


        for race in root.iter("Race"):
            places = []
            ids = []
            boxes = []
            splittimes = []
            split_margin = []
            weights = []
            margins = []
            split_position = []
            runtimes = []
            start_price = []
            dog_name = []

            for dog in race.iter("Dog"):
                if not dog.attrib["id"]:
                    continue
                if not dog[0].text:
                    continue
                if (dog[0].text in "RSBFNPTBD") or ("=" in dog[0].text):
                    # # print("scratched check working")
                    # print(dog)
                    # print(dog[0].text)
                    continue
                else:
                    places.append(dog[0].text)
                    ids.append(dog.attrib["id"])
                    boxes.append(dog[3].text)
                    splittimes.append(dog[13].text)
                    runtimes.append(dog[14].text)
                    weights.append(dog[5].text)
                    dog_name.append(dog[1].text)
                    if dog[6].text:
                        start_price.append(dog[6].text.replace("$","").replace("F",""))
                    if dog[0].text == "1":
                        margins.append(0)
                    else:
                        margins.append(dog[8].text)

            for i in range(len(splittimes)):
                if splittimes[0] == None:
                    split_margin.append(pd.NaT)
                    continue
                try:
                    split_margin.append(
                        "%.2f" % (float(splittimes[i]) - float(min(splittimes)))
                    )

                    # trying to get split position
                except:
                    split_margin.append(pd.NaT)

            zipall = list(
                zip(
                    places,
                    ids,
                    boxes,
                    splittimes,
                    split_margin,
                    runtimes,
                    weights,
                    margins,
                    start_price,
                    dog_name
                )
            )
            # print(zipall)
            for i in range(len(zipall)):
                zipall[i] = (
                    race.attrib["id"],
                    date.text,
                    track.text,
                    race[3].text,
                    race[4].text,
                ) + zipall[i]
                master_list.append(zipall[i])

    return master_list


In [5]:
master_list = result_getter(file_list_new, path)
#master_list

100%|██████████| 6405/6405 [00:49<00:00, 128.84it/s]


In [6]:
col_names = ["race_id", "date", "track","dist","grade", "place", "dog_id", "box", "split_times", "split_margins", "run_time", "weigth", "margin", "start_price", "dog_name"]
df = pd.DataFrame(master_list, columns = col_names)
df.place = df.place.astype(float)
df.start_price = df.start_price.astype(float)
df.place.unique()

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [7]:
print(df.size)
df = df.dropna()
#print(df2.size)
#df.to_csv("masterfile.csv")
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
with open("results-df.npy", "wb") as fp:   #Pickling
    
    pickle.dump(df, fp)

8212095


In [10]:
df.head()

,race_id,date,track,dist,grade,place,dog_id,box,split_times,split_margins,run_time,weigth,margin,start_price,dog_name
0,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,1.0,1296100098,2,6.73,0.00,26.17,30.2,0,10.6,SWIFT OPERATOR
1,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,2.0,1177320005,3,6.87,0.14,26.18,29.3,0.20,6.1,LAST PRINCESS
2,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,3.0,1549980026,5,6.99,0.26,26.26,29.4,1.24,11.9,DAINTREE PRINZ
3,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,4.0,1474670055,7,6.88,0.15,26.36,32.8,2.79,3.3,TIC TOC TOBY
4,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,5.0,913850094,8,6.91,0.18,26.42,32.4,3.60,16.0,RACE ROQUETTE


In [9]:
df.track.value_counts()

Ballarat         48727
Shepparton       48564
Warragul         48466
Bendigo          46188
Healesville      44860
Geelong          43661
Sale             30253
Cranbourne       30206
Warrnambool      28631
Traralgon        24520
The Meadows      22504
Sandown Park     21640
Meadows (MEP)    19477
Horsham          19354
Sandown (SAP)    19314
Name: track, dtype: int64

In [17]:
#generates adjusted place to transfer to form results

dfsorted = df.sort_values(['box'])
dfsorted['place_adj'] = 1/(dfsorted["place"])
grouped = dfsorted.groupby(['race_id'])
lens = []
for i,j in grouped:
    lens.append(len(j))


 

In [24]:
lens = pd.Series(lens)
lens.value_counts(normalize=True)

8     0.560244
7     0.255075
6     0.129615
5     0.042506
4     0.009141
3     0.001267
32    0.000940
28    0.000381
2     0.000354
20    0.000136
24    0.000123
16    0.000109
14    0.000054
1     0.000027
12    0.000027
dtype: float64

In [11]:
dfsorted.to_csv("masterfilenew.csv")


In [25]:
winners = df[df["place"]==1.0]